### Preparing Data

In [2]:
#imports
import os
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [3]:
#https://www.geeksforgeeks.org/how-to-iterate-over-files-in-directory-using-python/
# and https://stackoverflow.com/questions/20906474/import-multiple-csv-files-into-pandas-and-concatenate-into-one-dataframe



directory = 'data/movie_dataset/2_reviews_per_movie_raw'

dfs = list()

#Concatenate data into one pandas dataframe
for filename in os.listdir(directory):	
	data = pd.read_csv(os.path.join(directory, filename), header='infer')	
	dfs.append(data)
df = pd.concat(dfs, ignore_index=True)


df.head()



,username,rating,helpful,total,date,title,review
0,Imme-van-Gorp,7,102,123,30 January 2019,Unfortunately the ending ruined an otherwise ...,This movie is full of suspense. It makes you g...
1,sonofocelot-1,5,385,500,10 May 2016,...oh dear Abrams. Again.\n,I'll leave this review fairly concise. <br/><b...
2,mhodaee,5,110,143,4 August 2017,"Fantastic, gripping, thoroughly enjoyable, un...",I give the 5/10 out of the credit I owe to the...
3,fil-nik09,5,73,100,5 October 2016,Hmmm...\n,"First of all, I must say that I was expecting ..."
4,DVR_Brale,7,42,56,27 July 2016,Slow building & plot alternating claustrophob...,I've always loved movies with strong atmospher...


In [4]:
# retain only rating and review
df = df.drop(columns=['username', 'helpful', 'total', 'date','title'], errors='ignore')

#print unique ratings
df.rating.unique()


array(['7', '5', '9', '8', '10', 'Null', '6', '1', '4', '3', '2'],
      dtype=object)

In [5]:
#drop Null ratings
df = df[~df['rating'].str.contains('Null')]

# Convert "rating" to int
df= df.astype({'rating':'int'})
df.rating.unique()



array([ 7,  5,  9,  8, 10,  6,  1,  4,  3,  2])

In [6]:

# Drop rows where "rating" is NaN or reviews are missing
df = df.dropna(subset=['rating'])
df = df.dropna(subset=['review'])


df_ratings = df['rating']
df_reviews = df['review']


df_reviews.head()

0    This movie is full of suspense. It makes you g...
1    I'll leave this review fairly concise. <br/><b...
2    I give the 5/10 out of the credit I owe to the...
3    First of all, I must say that I was expecting ...
4    I've always loved movies with strong atmospher...
Name: review, dtype: object

In [47]:
df['review'] = df['review'].str.lower() 
df.head()

,rating,review
0,7,this movie is full of suspense. it makes you g...
1,5,i'll leave this review fairly concise. <br/><b...
2,5,i give the 5/10 out of the credit i owe to the...
3,5,"first of all, i must say that i was expecting ..."
4,7,i've always loved movies with strong atmospher...


In [48]:
#prep for train test split
df_reviews_train, df_reviews_test,  df_ratings_train_, df_ratings_test = \
    train_test_split(df_reviews, df_ratings, test_size=0.2, random_state=0)

In [49]:
#prep for train test split
df_train, df_test = \
    train_test_split(df, test_size=0.2, random_state=0)

In [38]:
#!pip install --upgrade keras-nlp
#!pip install --upgrade keras 

In [40]:
#import os

#os.environ["KERAS_BACKEND"] = "jax" 

#import keras_nlp 
#import tensorflow
#import keras

In [44]:
#!pip uninstall keras keras-core keras-nlp
#!pip install keras==2.15.0 keras-nlp==0.6.3

#!pip install --upgrade keras-nlp
#import os
os.environ["KERAS_BACKEND"] = "jax"  # Or "jax" or "torch"!

#from keras import keras_nlp
import tensorflow as tf
import keras

#TODO: I could pip install tensorflow but the following code is causing issues. I have added it to my PATH environment etc but its still a pain. See if you guys can get this to resolve.

The reason I introduced this code was to see how to persist the history of the models. This was one of my tasks.

In [45]:
multi_hot_layer = keras.layers.TextVectorization(
    max_tokens=4000, output_mode="multi_hot"
)
multi_hot_layer.adapt(df_train.map(lambda x, y: x))

multi_hot_ds = df_train.map(lambda x, y: multi_hot_layer(x), y)
multi_hot_val_ds = df_test.map(lambda x, y: (multi_hot_layer(x), y))

# We then learn a linear regression over that layer, and that's our entire
# baseline model!

inputs = keras.Input(shape=(4000,), dtype="int32")
outputs = keras.layers.Dense(1, activation="sigmoid")(inputs)
baseline_model = keras.Model(inputs, outputs)
baseline_model.compile(loss="binary_crossentropy", metrics=["accuracy"])
baseline_model.fit(multi_hot_ds, validation_data=multi_hot_val_ds, epochs=5)

AttributeError: 'DataFrame' object has no attribute 'map'